In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [0]:
text = text[:50000]

In [0]:
import pandas as pd
import numpy as np
import matplotlib as plt
from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from gensim import corpora
from gensim.models import Phrases
# from ds_voc.text_processing import TextProcessing
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [0]:
cleaned = []
def cleaning(text):
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)
#     convert to lower case
    tokens = [w.lower() for w in tokens]
#     remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
#     remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in stripped if not w in stop_words]
    # lemmatizing of words
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatize = ' '.join(lemmatizer.lemmatize(word) for word in words)
    return lemmatize
#     cleaned.append(stemmed)

In [0]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
cleaned = [cleaning(text)]

In [0]:
len(cleaned[0])

31280

In [0]:
from gensim.models import Word2Vec

word_model = Word2Vec(sentences = [cleaned[0].split()], size=100, min_count=1, 
                                    window=5, iter=100)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
# print('Checking similar words:')
# for word in ['model', 'network', 'train', 'learn']:
#   most_similar = ', '.join('%s (%.2f)' % (similar, dist) 
#                            for similar, dist in word_model.most_similar(word)[:8])
#   print('  %s -> %s' % (word, most_similar))

def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

Result embedding shape: (1751, 100)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """


In [0]:
pretrained_weights.shape

(1751, 100)

In [0]:
data = cleaned[0].split()

In [0]:
len(data)

4519

In [0]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, len(data) - seq_length, 1):
    dataX.append(data[i:i + seq_length])
    dataY.append(data[i + seq_length])

In [0]:
from keras.models import Sequential, Model
from keras.layers import Embedding


In [0]:

len(dataY)

4419

In [0]:
train_x = np.zeros([len(dataX), seq_length], dtype=np.int32)
train_y = np.zeros([len(dataX)], dtype=np.int32)
for i, sentence in enumerate(dataX):
  for t, word in enumerate(sentence):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(dataY[i])

In [0]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [0]:
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

# integer encode the documents
# encoded_docs= [one_hot(d[0], vocab_size) for d in enc]
# encoded_docs_y= [one_hot(d, vocab_size) for d in dataY]

# print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 100
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)
# define the model
models = Sequential()
models.add(Embedding(vocab_size, emdedding_size,weights=[pretrained_weights]))
models.add(LSTM(units=emdedding_size))
models.add(Dense(units=vocab_size))
models.add(Activation('softmax'))
models.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# summarize the model
# print(model.summary())
# fit the model
models.fit(train_x, train_y, epochs=50, verbose=2)
# # evaluate the model
# loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

Epoch 1/50
 - 18s - loss: 7.2618 - acc: 0.0154
Epoch 2/50
 - 16s - loss: 6.8785 - acc: 0.0199
Epoch 3/50
 - 16s - loss: 6.7417 - acc: 0.0226
Epoch 4/50
 - 16s - loss: 6.6530 - acc: 0.0260
Epoch 5/50
 - 16s - loss: 6.4542 - acc: 0.0310
Epoch 6/50
 - 16s - loss: 6.2463 - acc: 0.0333
Epoch 7/50
 - 16s - loss: 6.0346 - acc: 0.0398
Epoch 8/50
 - 16s - loss: 5.8307 - acc: 0.0412
Epoch 9/50
 - 16s - loss: 5.6269 - acc: 0.0505
Epoch 10/50
 - 16s - loss: 5.4381 - acc: 0.0561
Epoch 11/50
 - 16s - loss: 5.2452 - acc: 0.0695
Epoch 12/50
 - 16s - loss: 5.0650 - acc: 0.0783
Epoch 13/50
 - 16s - loss: 4.8774 - acc: 0.0935
Epoch 14/50
 - 16s - loss: 4.6971 - acc: 0.1143
Epoch 15/50
 - 16s - loss: 4.5173 - acc: 0.1428
Epoch 16/50
 - 16s - loss: 4.3377 - acc: 0.1774
Epoch 17/50
 - 16s - loss: 4.1587 - acc: 0.2084
Epoch 18/50
 - 16s - loss: 3.9830 - acc: 0.2548
Epoch 19/50
 - 16s - loss: 3.8046 - acc: 0.3039
Epoch 20/50
 - 16s - loss: 3.6305 - acc: 0.3394
Epoch 21/50
 - 16s - loss: 3.4567 - acc: 0.3777
E

In [0]:
t.shape

(100, 1)

In [0]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

def generate_next(text, num_generated=100):
#   print(text)
  word_idxs = [word2idx(word) for word in text]
  print(np.array(word_idxs).shape)
  for i in range(num_generated):
    prediction = models.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
generate_next(np.array(dataX[4]))

(100,)


'speak speak speak first citizen resolved rather die famish resolved resolved first citizen first know caius marcius chief enemy people knowt knowt first citizen let u kill corn price ist verdict talking nt let done away away second citizen one word good citizen first citizen accounted poor citizen patrician good authority surfeit would relieve u would yield u superfluity wholesome might guess relieved u humanely think dear leanness afflicts u object misery inventory particularise abundance sufferance gain let u revenge pike ere become rake god know speak hunger bread thirst revenge second citizen would proceed especially caius marcius first dog answer first citizen answer strike hath part wall shall action none wish wish crack devise silenced mummer doit womb nicelygawded ridge affection tenth butterfly safe fair volumnia last state fashion hark brother part dog miscarries bear menenius word volumnia good report senator come opinion coffin renown thousand beastly bear come messenger t